In [1]:
import pandas as pd
df = pd.read_parquet('order_item_review_ml.parquet')
df.shape
df.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,product_category_name,product_category_name_english,product_weight_g,product_length_cm,...,delay_vs_estimated_days,customer_city,customer_state,main_payment_type,max_installments,total_payment_value,review_score,review_creation_date,review_answer_timestamp,review_delay_days
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,cool_stuff,cool_stuff\r,650.0,28.0,...,9.0,campos dos goytacazes,RJ,credit_card,2.0,72.19,5,2017-09-21,2017-09-22 10:57:03,1.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,pet_shop,pet_shop\r,30000.0,50.0,...,3.0,santa fe do sul,SP,credit_card,3.0,259.83,4,2017-05-13,2017-05-15 11:34:13,1.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,moveis_decoracao,furniture_decor\r,3050.0,33.0,...,14.0,para de minas,MG,credit_card,5.0,216.87,5,2018-01-23,2018-01-23 16:06:31,1.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,perfumaria,perfumery\r,200.0,16.0,...,6.0,atibaia,SP,credit_card,2.0,25.78,4,2018-08-15,2018-08-15 16:39:01,1.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,ferramentas_jardim,garden_tools\r,3750.0,35.0,...,16.0,varzea paulista,SP,credit_card,3.0,218.04,5,2017-03-02,2017-03-03 10:54:59,1.0


In [2]:
#veamos todas las cols que tenemos:
df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'price',
       'freight_value', 'product_category_name',
       'product_category_name_english', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'delivery_time_days',
       'delay_vs_estimated_days', 'customer_city', 'customer_state',
       'main_payment_type', 'max_installments', 'total_payment_value',
       'review_score', 'review_creation_date', 'review_answer_timestamp',
       'review_delay_days'],
      dtype='object')

In [14]:
import numpy as np

# Target ordinal: 1–5
y = df["review_score"].astype("float32")


#saquemos las columnas que no voy a usar
cols_to_drop=[

              "review_score",
              "review_creation_date",
              "review_answer_timestamp",
              "review_delay_days",


              #Tengo la version simplificada delay vs estimated
              "order_purchase_timestamp",
              "order_approved_at",
              "order_delivered_carrier_date",
              "order_delivered_customer_date",
              "order_estimated_delivery_date",

              'customer_city',
              'customer_state',


              #Cardinalidad enorme que me romperia todo el arbol
              "order_item_id",
              "product_id",
              "order_id",






              "seller_id", #pruebo sacarlo

              #no poner seller_id aca me puede jugar una mala pasada, per son 3k. lo veo relevante (that means, seller_id stays)

]

#saquemos las columnas que no voy a usar
cols_to_drop_sellers=[

              "review_score",
              "review_creation_date",
              "review_answer_timestamp",
              "review_delay_days",


              #Tengo la version simplificada delay vs estimated
              "order_purchase_timestamp",
              "order_approved_at",
              "order_delivered_carrier_date",
              "order_delivered_customer_date",
              "order_estimated_delivery_date",

              'customer_city',
              'customer_state',


              #Cardinalidad enorme que me romperia todo el arbol
              "order_item_id",
              "product_id",
              "order_id",






              #no poner seller_id aca me puede jugar una mala pasada, per son 3k. lo veo relevante (that means, seller_id stays)

]


X=df.drop(columns=cols_to_drop,errors="ignore")

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

numeric_features = X.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numéricas:", numeric_features)
print("Categóricas:", categorical_features)


Numéricas: ['price', 'freight_value', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'delivery_time_days', 'delay_vs_estimated_days', 'max_installments', 'total_payment_value']
Categóricas: ['product_category_name', 'product_category_name_english', 'order_status', 'main_payment_type']


In [16]:
from sklearn.preprocessing import OrdinalEncoder #lo uso porque si no me queda una matriz densa, ya que histgradient no soporta otra manera.

preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), categorical_features),
    ]
)

In [17]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline

model = HistGradientBoostingRegressor(categorical_features="from_dtype", random_state=42)

pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model),
])

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")




# Evaluación como clasificación ordinal: redondeo a enteros 1–5
y_pred_round = np.clip(np.rint(y_pred), 1, 5).astype(int) #rint redondea lista, clip a numeros entre 1 y 5
y_test_int = y_test.astype(int)

accuracy = (y_pred_round == y_test_int).mean()
off_by_more_than1 = (np.abs(y_pred_round - y_test_int) > 1).mean()

print("Accuracy exacto:", accuracy)
print("Errores |pred - real| > 1:", off_by_more_than1)

MAE: 0.91
RMSE: 1.17
Accuracy exacto: 0.2475306656865898
Errores |pred - real| > 1: 0.12909450084991042


In [20]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

X_train_prep = pipe.named_steps["preprocess"].transform(X_train)
X_test_prep  = pipe.named_steps["preprocess"].transform(X_test)

dummy = DummyRegressor(strategy="median")
dummy.fit(X_train_prep, y_train)

y_dummy = dummy.predict(X_test_prep)

mae_dummy  = mean_absolute_error(y_test, y_dummy)
rmse_dummy = np.sqrt(mean_squared_error(y_test, y_dummy))

print("Dummy MAE:", mae_dummy)
print("Dummy RMSE:", rmse_dummy)

Dummy MAE: 0.908347487449646
Dummy RMSE: 1.618420853898408


Vamos a agregar info de seller copada, como el promedio de estrellas (nota: si solo hizo una venta, se menosprecia)

La idea es que como estoy usando un arbol, el modelo antes hacia

-es seller_id = 341 x --> no, y agarra ese camino

pero como los numeros de id son aleatorios, el orden que tienen no signifcan nada
dandonle este nuevo paradigma, podemos usar esa info para predecir mejor.

In [36]:
import pandas as pd
df = pd.read_parquet('order_item_review_ml.parquet')
df.shape
df.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,product_category_name,product_category_name_english,product_weight_g,product_length_cm,...,delay_vs_estimated_days,customer_city,customer_state,main_payment_type,max_installments,total_payment_value,review_score,review_creation_date,review_answer_timestamp,review_delay_days
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,cool_stuff,cool_stuff\r,650.0,28.0,...,9.0,campos dos goytacazes,RJ,credit_card,2.0,72.19,5,2017-09-21,2017-09-22 10:57:03,1.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,pet_shop,pet_shop\r,30000.0,50.0,...,3.0,santa fe do sul,SP,credit_card,3.0,259.83,4,2017-05-13,2017-05-15 11:34:13,1.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,moveis_decoracao,furniture_decor\r,3050.0,33.0,...,14.0,para de minas,MG,credit_card,5.0,216.87,5,2018-01-23,2018-01-23 16:06:31,1.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,perfumaria,perfumery\r,200.0,16.0,...,6.0,atibaia,SP,credit_card,2.0,25.78,4,2018-08-15,2018-08-15 16:39:01,1.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,ferramentas_jardim,garden_tools\r,3750.0,35.0,...,16.0,varzea paulista,SP,credit_card,3.0,218.04,5,2017-03-02,2017-03-03 10:54:59,1.0


In [37]:
df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'price',
       'freight_value', 'product_category_name',
       'product_category_name_english', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'delivery_time_days',
       'delay_vs_estimated_days', 'customer_city', 'customer_state',
       'main_payment_type', 'max_installments', 'total_payment_value',
       'review_score', 'review_creation_date', 'review_answer_timestamp',
       'review_delay_days'],
      dtype='object')

In [38]:
cols_to_drop_sellers=[

              "review_score",
              "review_creation_date",
              "review_answer_timestamp",
              "review_delay_days",


              #Tengo la version simplificada delay vs estimated
              "order_purchase_timestamp",
              "order_approved_at",
              "order_delivered_carrier_date",
              "order_delivered_customer_date",
              "order_estimated_delivery_date",

              'customer_city',
              'customer_state',


              #Cardinalidad enorme que me romperia todo el arbol
              "order_item_id",
              "product_id",
              "order_id",


]

X=df.drop(columns=cols_to_drop_sellers,errors="ignore")

In [40]:
from sklearn.preprocessing import TargetEncoder

from sklearn.preprocessing import TargetEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = X_train.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

# Marcamos seller_id como alta cardinalidad
high_cardinality = ["seller_id"]
other_cat = [c for c in categorical_features if c not in high_cardinality]

preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),

        # Target encoding para seller_id
        ("target_enc", TargetEncoder(
            target_type="continuous",
            smooth="auto",   # o un valor numérico si quiere fijarlo
            cv=5,            # número de folds internos para el cross-fitting
            random_state=42,
        ), high_cardinality),

        # Ordinal para el resto de categóricas
        ("ordinal_cat", OrdinalEncoder(
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ), other_cat),
    ]
)

model = HistGradientBoostingRegressor(random_state=42)

pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model),
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")




# Evaluación como clasificación ordinal: redondeo a enteros 1–5
y_pred_round = np.clip(np.rint(y_pred), 1, 5).astype(int) #rint redondea lista, clip a numeros entre 1 y 5
y_test_int = y_test.astype(int)

accuracy = (y_pred_round == y_test_int).mean()
off_by_more_than1 = (np.abs(y_pred_round - y_test_int) > 1).mean()

print("Accuracy exacto:", accuracy)
print("Errores |pred - real| > 1:", off_by_more_than1)

MAE: 0.90
RMSE: 1.16
Accuracy exacto: 0.2772545596545229
Errores |pred - real| > 1: 0.13217255478476592


In [44]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

X_train_prep = pipe.named_steps["preprocess"].transform(X_train)
X_test_prep  = pipe.named_steps["preprocess"].transform(X_test)

dummy = DummyRegressor(strategy="median")
dummy.fit(X_train_prep, y_train)

y_dummy = dummy.predict(X_test_prep)

mae_dummy  = mean_absolute_error(y_test, y_dummy)
rmse_dummy = np.sqrt(mean_squared_error(y_test, y_dummy))

print("Dummy MAE:", mae_dummy)
print("Dummy RMSE:", rmse_dummy)

Dummy MAE: 0.908347487449646
Dummy RMSE: 1.618420853898408


In [45]:
from sklearn import set_config
set_config(display='diagram')
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['price', 'freight_value',
                                                   'product_weight_g',
                                                   'product_length_cm',
                                                   'product_height_cm',
                                                   'product_width_cm',
                                                   'delivery_time_days',
                                                   'delay_vs_estimated_days',
                                                   'max_installments',
                                                   'total_payment_value']),
                                                 ('target_enc',
                                                  TargetEncoder(random_state=42,
                                                                target_type='continuous'),
                                                  ['seller_id']),
                                                 ('ordinal_cat',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['product_category_name',
                                                   'product_category_name_english',
                                                   'order_status',
                                                   'main_payment_type'])])),
                ('model', HistGradientBoostingRegressor(random_state=42))])